In [13]:
num_cores=16

from tune_sklearn import TuneGridSearchCV, TuneSearchCV
from sklearn.model_selection import (
    GridSearchCV,
    RepeatedKFold,
    LeaveOneOut,
    RepeatedStratifiedKFold,
)

from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, make_scorer

from imblearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skorch import NeuralNetBinaryClassifier, NeuralNetClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from functools import partial

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Read the DataFrame and Series from the CSV files
X = pd.read_csv("X.csv")
y = pd.read_csv("y.csv", squeeze=True, header=None)

/var/tmp/pbs.7507837.pbs/ipykernel_2060235/2846834199.py:38: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  y = pd.read_csv("y.csv", squeeze=True, header=None)


In [16]:
torch.manual_seed(42)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

   be_alcohol_freq.0.0  be_physact_walks.0.0  be_smok_packyears.0.0  \
0                 6.00                 40.92                  0.000   
1                 0.15                 30.00                  0.000   
2                 3.50                 65.58                 30.647   
3                 6.00                 15.00                  0.000   
4                 0.00                 15.00                 24.500   

   bio_alb.0.0  bio_alp.0.0  bio_alt.0.0  bio_ast.0.0  bio_bas.0.0  \
0        47.06         75.7        57.04         37.3         0.02   
1        47.90        123.6        33.65         27.5         0.00   
2        48.89        185.4        28.27         46.1         0.00   
3        43.32        118.6        25.55         32.0         0.00   
4        42.44         73.3        40.87         36.3         0.00   

   bio_chol.0.0  bio_cr.0.0  ...  bp_sys_avg  bp_dia_avg  ethnicity.asian  \
0         4.156        63.4  ...       151.5        88.5              0.0  

In [25]:
from skorch.callbacks import EarlyStopping

torch.manual_seed(42)

# Neural network architecture with dropout
class NeuralNet(nn.Module):
    def __init__(
        self, dropout_prob=0, n_neurons1=8, n_neurons2=8, weight_constraint=1.0
    ):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, n_neurons1)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.layer2 = nn.Linear(n_neurons1, n_neurons2)
        self.dropout2 = nn.Dropout(dropout_prob)
        self.layer3 = nn.Linear(n_neurons2, 1)
        self.weight_constraint = weight_constraint

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)
        x = torch.relu(self.layer2(x))
        x = self.dropout2(x)
        x = self.layer3(x)  # Removed torch.sigmoid from the last layer
        return x


X_train_resampled, y_train_resampled = X_train, y_train
input_size = X_train_resampled.shape[1]
X_train_resampled = torch.tensor(X_train_resampled.values, dtype=torch.float32)
y_train_resampled = torch.tensor(y_train_resampled.values, dtype=torch.float32)

def tofloat32(x):
#    return x.astype(np.float32)
    return torch.tensor(x, dtype=torch.float32).to(device)

model = NeuralNetBinaryClassifier(
    NeuralNet,
    criterion=nn.BCEWithLogitsLoss,  # Changed to BCEWithLogitsLoss
    batch_size=64,
    max_epochs=1000,
    verbose=False,
    optimizer=optim.Adam,
    optimizer__lr=0.001,
    callbacks=[EarlyStopping(patience=5)],
    device=device,
)

#model.to(device)

cont_vars = [
    0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    20,
    21,
    22,
    23,
    24,
    26,
    27,
    28,
    46,
    47,
    48,

]
preprocessor = ColumnTransformer(
    transformers=[("scaler", StandardScaler(), cont_vars)], remainder="passthrough"
)

smote = SMOTE(sampling_strategy=0.5, random_state=42)
under = RandomUnderSampler(sampling_strategy=1, random_state=42)
over = RandomOverSampler(random_state=42)
# smoteenn = SMOTEENN(random_state=42)
# steps = [('under', under), ('model', model)]


# Create a list of param_grids
param_grids = [
    {
        "model__optimizer": [
            optim.Adam,
            optim.Adamax,
            optim.NAdam,
        ],
    },
    {
        "model__optimizer__lr": [
            0.00001,
            0.000025,
            0.00005,
            0.0001,
            0.0005,
            0.001,
            0.005,
            0.01,
        ],
    },
    #{
    #    "model__batch_size": [8, 16, 32, 64, 128, 256, 512],
    #    "model__max_epochs": [20, 50, 150, 200, 250, 400, 600],
    #},
    {
        "model__module__n_neurons1": [1, 2, 4, 8, 16, 32],
        "model__module__n_neurons2": [1, 2, 4, 8, 16, 32],
    },
    {
        "model__module__dropout_prob": [0.0, 0.2, 0.4, 0.6, 0.8],
    },
]


cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=100, random_state=42)  # changed
roc_auc_score_2 = make_scorer(roc_auc_score, needs_threshold=False)

# Initialize the best_params dictionary
best_params = {}

In [26]:
# Iterate through param_grids
for param_grid in param_grids:
    steps = [
        ("scale", preprocessor),
        # ('smoteenn', smoteenn),
        # ('over', over),
        ("smote", smote),
        # ('tofloat32_', FunctionTransformer(tofloat32, accept_sparse=True)),
        ("under", under),
        ("tofloat32", FunctionTransformer(tofloat32, accept_sparse=True)),
        ("model", model),
    ]
    pipeline = Pipeline(steps=steps)

    print(model.get_params())
    
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        #search_optimization='hyperopt',
        #random_state=42,
        scoring=roc_auc_score_2,
        n_jobs=-1,
        cv=cv,
        verbose=0,
        #use_gpu=True,
    )    

    grid_result = grid.fit(X_train_resampled, y_train_resampled)

    # Update the model with the best parameters
    for key, value in grid_result.best_params_.items():
        setattr(model, key.split("__")[-1], value)

    # Print the best parameters and score
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_["mean_test_score"]
    stds = grid_result.cv_results_["std_test_score"]
    params = grid_result.cv_results_["params"]
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    # Update the best_params dictionary
    best_params.update(grid_result.best_params_)

# Print the final best_params dictionary
print("Final best parameters:", best_params)



{'module': <class '__main__.NeuralNet'>, 'criterion': <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.01, 'max_epochs': 1000, 'batch_size': 64, 'iterator_train': <class 'torch.utils.data.dataloader.DataLoader'>, 'iterator_valid': <class 'torch.utils.data.dataloader.DataLoader'>, 'dataset': <class 'skorch.dataset.Dataset'>, 'train_split': <skorch.dataset.ValidSplit object at 0x154a48af3250>, 'callbacks': [<skorch.callbacks.training.EarlyStopping object at 0x154b4c1977f0>], 'predict_nonlinearity': 'auto', 'warm_start': False, 'verbose': False, 'device': device(type='cpu'), '_params_to_validate': {'optimizer__lr'}, 'optimizer__lr': 0.001, 'threshold': 0.5, 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer object at 0x154a3be918e0>, 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring object at 0x154a3be91910>, 'callbacks__train_loss__name': 'train_loss', 'callbacks__train_loss__lower_is_better': True, '

In [21]:
grid_result.best_estimator_

Pipeline(steps=[('scale',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', StandardScaler(),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12, 13, 14, 15, 16,
                                                   17, 18, 20, 21, 22, 23, 24,
                                                   26, 27, 28, 46, 47, 48])])),
                ('smote', SMOTE(random_state=42, sampling_strategy=0.5)),
                ('under',
                 RandomUnderSampler(random_state=42, sampling_strategy=1)),
                ('tofloat32',
                 FunctionTransformer(ac...
                                     func=<function tofloat32 at 0x154a43380550>)),
                ('model',
                 <class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=NeuralNet(
    (layer1): Linear(in_features=57, out_features=8,